In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score  #交叉验证评分
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import mean_squared_error #均方根误差
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv('../data/train_chuli.csv')

In [16]:
# 输出数据预览
print(dataset.head())

   PassengerId  Survived  Pclass  Sex   Age  SibSp  Parch     Fare  \
0            1         0       3    1  22.0      1      0   7.2500   
1            2         1       1    0  38.0      1      0  71.2833   
2            3         1       3    0  26.0      0      0   7.9250   
3            4         1       1    0  35.0      1      0  53.1000   
4            5         0       3    1  35.0      0      0   8.0500   

   family_size  Embarked_C  Embarked_Q  Embarked_S  
0            2           0           0           1  
1            2           1           0           0  
2            1           0           0           1  
3            2           0           0           1  
4            1           0           0           1  


In [17]:
from sklearn.model_selection import train_test_split #划分数据集
# 准备训练数据
x = dataset.drop(['Survived'], axis = 1)
y = dataset['Survived']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.3,random_state=0) 
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(623, 11) (268, 11) (623,) (268,)


In [18]:
# !pip install catboost
import catboost as cb
from catboost import CatBoostClassifier

In [19]:
from matplotlib.cm import rainbow 

#决策树
rf_scores = []

rf_classifier = CatBoostClassifier(iterations=1500, learning_rate=0.005)
rf_classifier.fit(x_train, y_train)

0:	learn: 0.6898705	total: 4.96ms	remaining: 7.44s
1:	learn: 0.6863698	total: 10.3ms	remaining: 7.73s
2:	learn: 0.6833087	total: 14.4ms	remaining: 7.2s
3:	learn: 0.6801450	total: 18.9ms	remaining: 7.07s
4:	learn: 0.6770477	total: 24.2ms	remaining: 7.24s
5:	learn: 0.6741230	total: 28.3ms	remaining: 7.06s
6:	learn: 0.6709372	total: 33.3ms	remaining: 7.1s
7:	learn: 0.6680397	total: 39.7ms	remaining: 7.4s
8:	learn: 0.6663305	total: 42.1ms	remaining: 6.98s
9:	learn: 0.6635632	total: 46.6ms	remaining: 6.95s
10:	learn: 0.6607393	total: 55.3ms	remaining: 7.49s
11:	learn: 0.6580116	total: 59.8ms	remaining: 7.41s
12:	learn: 0.6550468	total: 63.9ms	remaining: 7.31s
13:	learn: 0.6521816	total: 69.8ms	remaining: 7.41s
14:	learn: 0.6501918	total: 74ms	remaining: 7.33s
15:	learn: 0.6479269	total: 78.4ms	remaining: 7.27s
16:	learn: 0.6452955	total: 84.5ms	remaining: 7.37s
17:	learn: 0.6425830	total: 88.9ms	remaining: 7.32s
18:	learn: 0.6398624	total: 93.3ms	remaining: 7.27s
19:	learn: 0.6373941	total:

CatBoostClassifier
- iterations：指定梯度提升的迭代次数，也就是基础模型的数量。默认值为 1000。
- learning_rate：每个基础模型的学习率，用于缩放每次梯度下降的步长。默认值为 0.03。
- depth：每个基础模型的深度，控制了每棵树的最大深度。默认值为 6。
- l2_leaf_reg：L2 正则化参数，用于减少模型的过拟合。默认值为 3。
- random_seed：随机数生成器的种子，用于复现结果。默认为 None。
- early_stopping_rounds：当验证数据集的损失函数不再下降时，停止训练的迭代次数。默认值为 None，表示不启用早停。
- verbose：控制训练期间的日志输出。默认值为 0，表示不输出日志。
- loss_function：指定分类问题的损失函数。默认值为 'Logloss'，即逻辑斯蒂回归损失函数。
- eval_metric：用于度量模型性能的指标。默认为 'Accuracy'。

In [20]:
#评价指标
import numpy as np

# 计算混淆矩阵
def compute_confusion_matrix(precited,expected):
    part = precited ^ expected             # 对结果进行分类，亦或使得判断正确的为0,判断错误的为1
    pcount = np.bincount(part)             # 分类结果统计，pcount[0]为0的个数，pcount[1]为1的个数
    tp_list = list(precited & expected)    # 将TP的计算结果转换为list
    fp_list = list(precited & ~expected)   # 将FP的计算结果转换为list
    tp = tp_list.count(1)                  # 统计TP的个数
    fp = fp_list.count(1)                  # 统计FP的个数
    tn = pcount[0] - tp                    # 统计TN的个数
    fn = pcount[1] - fp                    # 统计FN的个数
    return tp, fp, tn, fn

# 计算常用指标
def compute_indexes(tp, fp, tn, fn):
    accuracy = (tp+tn) / (tp+tn+fp+fn)     # 准确率
    precision = tp / (tp+fp)               # 精确率
    recall = tp / (tp+fn)                  # 召回率
    F1 = (2*precision*recall) / (precision+recall)    # F1
    return accuracy, precision, recall, F1


In [21]:
rfc_score = rf_classifier.score(x_test, y_test)#测试集准确率,j即accuracy
expected = y_test.astype('int32')#预期值
# print(type(expected))
# print(expected)
rfc_pre=rf_classifier.predict(x_test)
precited = rfc_pre.astype('int32')#模型预测值
# print(type(precited))
tp, fp, tn, fn = compute_confusion_matrix(precited, expected)
accuracy, precision, recall, F1 = compute_indexes(tp, fp, tn, fn)
rfc_pre = rf_classifier.predict(x_test)
rfc_auc = metrics.roc_auc_score(y_test, rfc_pre)
rfc_rmse = np.sqrt(mean_squared_error(y_test, rfc_pre))
print('TP FP TN FN')
print('%d %d %d %d'%(tp,fp,tn,fn))
print('accuracy precision recall F1')
print('%.4lf %.4lf %.4lf %.4lf'%(accuracy, precision, recall, F1))
print('auc rmse')
print('%.4lf %.4lf'%(rfc_auc, rfc_rmse))

TP FP TN FN
75 18 150 25
accuracy precision recall F1
0.8396 0.8065 0.7500 0.7772
auc rmse
0.8214 0.4006
